# Building Systems with the ChatGPT API

ChatGPT API를 사용하여 시스템을 구축하는 과정에 오신 것을 환영합니다.

시스템을 구축하는 과정에는 단일 프롬프트나 대규모 언어 모델에 대한 단일 호출보다 훨씬 더 많은 것이 필요합니다.
여기에서 LLM을 사용하여 복잡한 애플리케이션을 구축하기 위한 모범 사례를 공유하고자 합니다. 우리는 언어 모델에 대한 여러 호출을 연결하고 이전 호출의 출력에 따라 다른 명령을 사용하고 때로는 외부 소스에서 항목을 조회하는 종단 간 고객 지원 시스템을 구축하는 실행 예제를 사용했습니다.

예를 들어 "판매 중인 TV에 대해 알려주세요."와 같은 사용자 입력이 주어지면 다음 단계를 사용하여 이를 처리합니다.
먼저 **입력 내용을 평가**하여 증오심 표현과 같은 문제가 있는 콘텐츠가 포함되어 있지 않은지 확인할 수 있습니다. 다음으로 시스템은 **입력을 처리**합니다. 이것이 **어떤 유형의 쿼리인지 식별**합니다. 불만 또는 제품 정보 요청 등입니까? 제품 문의라고 판단되면 TV에 대한 관련 정보를 검색하고 언어 모델을 사용하여 유용한 답변을 작성합니다.

마지막으로 **출력을 확인**하여 부정확하거나 부적절한 답변과 같은 문제가 없는지 확인합니다. 이 과정에서 확인할 수 있는 주제는 **응용 프로그램이 종종 최종 사용자에게 보이지 않는 여러 내부 단계를 필요로 한다는 것입니다**.

#  Language Models, the Chat Format and Tokens

## Large Language Model

##### Text generation process

| I love eating |  -------------------------------------------------- |
| ------------------|-------------------|
| [prompt] | bagles with cream cheese |
|        | my mothers's meatloaf |
|        | out with friends |

##### Supervised Learning (x --> y)

Restaurant reviewers sentiment classification

| Input x |  Output y |
| ------------------|-------------------|
| The pastromi sandwich was great | positive |
| Service was slow and the food was so~so | netative |
| The earl grey tea was fantastic. | positive |
| [Best pizza I've aver had!] | [positive] |


Get labeled data >> Train AI model on data >> Deploy & vall model

##### LLM : How it works

A language model is built by using supervised learning(x->y) to repeatedly predict the next word.

My favorite food is a bagel with cream cheese and lox.

| Input x |  Output y |
| ------------------|-------------------|
| My favorite food is a | bagel |
| My favorite food is a bagel | with |
| My favorite food is a bagel with | cream |
| My favorite food is a bagel with cream | cheese |


##### Two types of large language models (LLMs)

**Base LLM**

Predicts next word, based on text training data

*[Once up a time, there was a unicorn]*
<br>
that lived in a magical forest with all her unicorn friends.
<br>
<br>
*[What is the capital of France?]*
<br>
What is France's largest city?
<br>
What is France's population?
<br>
What is the currency of France?


**Instruction Tuned LLM**

Tries to follow instructions

*[What is teh capital of France?]*
<br>
The capital of France is Paries.

**Getting from a Base LLM to an Instruction Tuned LLM**
1. Train a Base LLm on a lot of data.
2. Further train the model:
  * Fine-tune on examples of where the output follows on input instruction.
  * Obtain human-ratings of the quality of different LLM outputs, on criteria such as whether it is helpfun, honest and harmless
  * Tune LLM to increase probability that it generates the more highly rated outputs (using RLHF: Reinforcement Learning from Human Feedback)

## Setup
#### Load the API key and relevant Python libaries.
이 과정에서는 OpenAI API 키를 로드하는 몇 가지 코드를 제공했습니다.

In [1]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

#### helper function
This may look familiar if you took the earlier course "ChatGPT Prompt Engineering for Developers" Course

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

## Prompt the model and get a completion

In [3]:
response = get_completion("What is the capital of France?")

print(response)

## Tokens

In [4]:
response = get_completion("Take the letters in lollipop \
and reverse them")
print(response)

The reversed letters of "lollipop" are "pillipol".


In [5]:
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")
print(response)

p-o-p-i-l-l-o-l


<img src="https://media.licdn.com/dms/image/D4D22AQH841LuwixZwg/feedshare-shrink_2048_1536/0/1687512301353?e=1691625600&v=beta&t=tl9hUuILCCsLXjSvUvE9W_mDAFV1_3lkhfHZ8w0FfUc" width=400>

For English language input, 1 token is around 4 characters, or 3/4 of a word.

Token LImits
* Different models have different limit on the number  tokens in the input "context"+ output completion
* gpt3.5-turbo ~ 4000 tokens

### System, User and Assistant Messages
```
messages =
[
 {"role": "system",
  "content": "You are an assistant ..."},
 {"role": "user",
  "content": "Tell me a joke "},
 {"role": "assistant",
  "content": "Why did the chicken ..."},
  ...
```
<img src="mesg.PNG" width=350>


## Helper function (chat format)
Here's the helper function we'll use in this course.

In [6]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message["content"]

In [7]:
messages =  [  
{'role':'system', 
 'content':"You are an assistant who responds \
            in the style of Dr Seuss."},    
{'role':'user', 
 'content':"write me a very short poem about a happy carrot"},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

In a garden so bright, a carrot did grow,
With a leafy green top and a vibrant orange glow.
Oh, how the carrot would dance and cheer,
Filled with joy, bringing smiles far and near!

With each sunny day and each drop of rain,
The carrot would grow, free from any strain.
Its sweetness and crunch, a flavor so divine,
Bringing happiness to all, it was simply sublime!

With a hop and a skip, the carrot would prance,
Spreading laughter and giggles, in every chance.
From salad to stew, it brought delight to each bite,
A happy carrot, shining so bright!

So let's celebrate this veggie, so jolly and merry,
A symbol of happiness, let's not tarry.
For a happy carrot, with its delightful flair,
Brings joy to our plates, and a smile we gladly wear!


In [8]:
# length
messages =  [  
{'role':'system',
 'content':'All your responses must be one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

Once upon a time, there was a cheerful carrot named Charlie who lived in a vibrant garden.


In [9]:
# combined
messages =  [  
{'role':'system',
 'content':"You are an assistant who \
responds in the style of Dr Seuss. \
All your responses must be one sentence long."},    
{'role':'user',
 'content':"write me a story about a happy carrot"},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

Once there was a happy carrot named Barry who grew tall and orange in the garden.


In [15]:
def get_completion_and_token_count(messages, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0, 
                                   max_tokens=500):
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message["content"]
    
    token_dict = {
'prompt_tokens':response['usage']['prompt_tokens'],
'completion_tokens':response['usage']['completion_tokens'],
'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

In [17]:
messages = [
{'role':'system', 
 'content':"You are an assistant who responds in the style of Dr Seuss."},    
{'role':'user',
 'content':"write me a very short poem about a happy carrot"},  
] 
response, token_dict = get_completion_and_token_count(messages)

In [18]:
print(response)

Oh, the happy carrot, so bright and orange,
Grown in the garden, a joyful forage.
With a smile so wide, from top to bottom,
It fills our hearts with a happy autumn.

In the soil it grew, with love and care,
Nourished by sunshine, fresh air to share.
Its leaves so green, waving in the breeze,
A happy carrot, it aims to please.

With a crunch and a munch, it brings delight,
A healthy snack, oh what a sight!
From garden to table, it brings us glee,
The happy carrot, so full of glee.

So let's celebrate this veggie so grand,
With a happy carrot in each hand.
For in its sweetness, we find pure bliss,
The happy carrot, a simple, joyful kiss.


In [19]:
print(token_dict)

{'prompt_tokens': 35, 'completion_tokens': 162, 'total_tokens': 197}
